## Creating a dataframe with coordinates, raster values and region from a shapefile

In [1]:
## Function to get raster pixel coordinates, values, and region names from a shapefile
get_raster_coordinates_and_regions <- function(raster_data, shapefile_data) {
  
  # Convert raster data to sf point dataframe with longitude and latitude
  raster_points <- rasterToPoints(raster_data) %>% data.frame() 
  
  raster_points_sp <- st_as_sf(raster_points, coords = c("x", "y"), crs = st_crs(shapefile_data)) %>% as("Spatial")

  # Find corresponding region for each point
  points_regions <- sp::over(raster_points_sp, shapefile_data)

  # Add coordinates, values, and NAMES to the data frame
  result_df <- points_regions %>% 
    select(NAME_1, NAME_2, NAME_3) %>%
    mutate(longitude = raster_points[,2], latitude = raster_points[,3], values = raster_points[,1])


  return(result_df)
}

In [ ]:
#+++FUNCTION USAGE+++

# Load necessary libraries
library(raster)
library(sf)
library(geodata)
library(tidyverse)
library(sp)

## Load the data
* Raster data - Eg prevalence map, environmental variable maps etc.
* Shapefile - Eg. polygon of a country, region, district etc.

In [3]:
# Set the directory for data
output_dir <- "data/"

# Load raster data
elevation_ETH <- elevation_30s(country = "ETH", path = output_dir)

elevation_ETH_raster <- raster(elevation_ETH)

# Load shapefile data
ethiopia_boundary <- gadm("ETH", level = 3, path = output_dir) %>% st_as_sf() %>% as("Spatial") 


In [4]:
# Example usage of the function with the provided inputs
raster_data <-  elevation_ETH_raster # Replace with your raster file
shapefile_data <- ethiopia_boundary      # Replace with your shapefile

output_data <- get_raster_coordinates_and_regions(raster_data, shapefile_data)
print(output_data)

      NAME_1           NAME_2            NAME_3 longitude latitude   values
1       <NA>             <NA>              <NA>  14.89583      930 37.92083
2       <NA>             <NA>              <NA>  14.88750     1006 37.91250
3       <NA>             <NA>              <NA>  14.88750      922 37.92083
4       <NA>             <NA>              <NA>  14.88750     1050 37.92917
5       <NA>             <NA>              <NA>  14.87917     1005 37.91250
6       <NA>             <NA>              <NA>  14.87917      921 37.92083
7       <NA>             <NA>              <NA>  14.87917      965 37.92917
8       <NA>             <NA>              <NA>  14.87917      939 37.93750
9       <NA>             <NA>              <NA>  14.87083     1004 37.90417
10      <NA>             <NA>              <NA>  14.87083     1029 37.91250
11      <NA>             <NA>              <NA>  14.87083      960 37.92083
12      <NA>             <NA>              <NA>  14.87083      927 37.92917
13      <NA>

## Comments
* For the pixels along the borders, the script is unable to extract the names for the regions. However, all the pixels for each region are extracted correctly. 